In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split


 Preprocessing

In [2]:
# Load the dataset
data = pd.read_csv(r'D:\Week4\data\store.csv')
df = pd.read_csv(r'D:\Week4\data\train.csv')

# Check the columns of the DataFrame
print("Columns in the DataFrame:")
print(data.columns)
print("Columns in the DataFrame:")
print(df.columns)

# Inspect the first few rows of the DataFrame
print("First few rows of the DataFrame:")
print(data.head())
print("First few rows of the DataFrame:")
print(df.head())
# Handle missing values in CompetitionOpenSinceMonth and CompetitionOpenSinceYear
data['CompetitionOpenSinceMonth'] = data['CompetitionOpenSinceMonth'].fillna(1)  # Default to January if NaN
data['CompetitionOpenSinceYear'] = data['CompetitionOpenSinceYear'].fillna(2000)  # Default to year 2000 if NaN

# Create a new date column from CompetitionOpenSinceMonth and CompetitionOpenSinceYear
data['CompetitionOpenSince'] = pd.to_datetime(
    data['CompetitionOpenSinceMonth'].astype(int).astype(str) + '-' +
    data['CompetitionOpenSinceYear'].astype(int).astype(str),
    format='%m-%Y',
    errors='coerce'
)

# Check the updated DataFrame with CompetitionOpenSince
print("Updated DataFrame with CompetitionOpenSince:")
print(data.head())

# Create a date column for analysis (example: using today's date or a specific date)
data['date'] = pd.to_datetime('2023-09-23')  # Replace with your actual date logic

# If you have a holiday_date column, ensure it is present
data['holiday_date'] = pd.to_datetime('2023-12-25')  # Example holiday date

# Create additional date-related features
data['weekday'] = data['date'].dt.weekday
data['weekend'] = (data['weekday'] >= 5).astype(int)
data['days_to_holiday'] = (data['holiday_date'] - data['date']).dt.days
data['days_after_holiday'] = (data['date'] - data['holiday_date']).dt.days
data['beginning_of_month'] = data['date'].dt.day <= 7
data['mid_month'] = (data['date'].dt.day > 7) & (data['date'].dt.day <= 14)
data['end_of_month'] = data['date'].dt.day > 14

# Fill NaN values
data.fillna(0, inplace=True)  # Example of filling NaNs

# Standardize numeric features
scaler = StandardScaler()
numeric_features = data.select_dtypes(include=['float64', 'int64'])
data[numeric_features.columns] = scaler.fit_transform(numeric_features)

# Check the updated DataFrame
print("Updated DataFrame:")
print(data.head())

Columns in the DataFrame:
Index(['Store', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'],
      dtype='object')
Columns in the DataFrame:
Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday'],
      dtype='object')
First few rows of the DataFrame:
   Store StoreType Assortment  CompetitionDistance  CompetitionOpenSinceMonth  \
0      1         c          a               1270.0                        9.0   
1      2         a          a                570.0                       11.0   
2      3         a          a              14130.0                       12.0   
3      4         c          c                620.0                        9.0   
4      5         a          a              29910.0                        4.0   

   CompetitionOpenSinceYear  Promo2  Promo2SinceWeek  Promo2Sinc

In [3]:
# Check for missing values
print(df.isnull().sum())

# Handle missing values if necessary
# Fill missing values in 'Customers' and assign it back to the column
df['Customers'] = df['Customers'].fillna(0)

# Convert 'Date' to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Extract additional features
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['DayOfWeek'] = df['Date'].dt.dayofweek

# Define features and target
X = df.drop(['Sales'], axis=1)  # Features
y = df['Sales']  # Target

# Convert categorical variables to numeric if needed
X = pd.get_dummies(X, columns=['Store', 'StateHoliday'], drop_first=True)

# Ensure all data is numeric
X = X.select_dtypes(exclude=['object'])  # Keep only numeric columns

from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Train a Random Forest model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64


In [31]:

X = df.drop(['sales'], axis=1)  # Features
y = df['sales']  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline = Pipeline([
    ('model', RandomForestRegressor())
])

pipeline.fit(X_train, y_train)

KeyError: "['sales'] not found in axis"